In [16]:
!pip install scipy
!pip install polars

     --------------------------------------- 27.3/27.3 MB 28.5 MB/s eta 0:00:00


In [85]:
import pandas as pd
import numpy as np
import polars as pl
from scipy.stats import shapiro

In [118]:
eth1 = pl.read_csv("BAZAR_KHAFAJI_ETHSD_AUG2017_APR2024.csv", infer_schema_length=2000)\
    .select(pl.col("timestamp"), pl.col("open"), pl.col("high"), pl.col("low"), pl.col("close"), pl.col("volume"), pl.col("return"))\
    .fill_nan(None)\
    .drop_nulls()\

print(eth1)

#load eth1 csv file and display

shape: (554_952, 7)
┌────────────┬─────────┬─────────┬─────────┬─────────┬────────────┬────────┐
│ timestamp  ┆ open    ┆ high    ┆ low     ┆ close   ┆ volume     ┆ return │
│ ---        ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---        ┆ ---    │
│ i64        ┆ f64     ┆ f64     ┆ f64     ┆ f64     ┆ f64        ┆ f64    │
╞════════════╪═════════╪═════════╪═════════╪═════════╪════════════╪════════╡
│ 1505631900 ┆ 244.1   ┆ 245.34  ┆ 244.04  ┆ 244.58  ┆ 125.832853 ┆ 0.48   │
│ 1507087800 ┆ 292.84  ┆ 293.28  ┆ 292.84  ┆ 293.28  ┆ 45.554642  ┆ 0.44   │
│ 1507131300 ┆ 292.95  ┆ 292.95  ┆ 292.92  ┆ 292.94  ┆ 24.737537  ┆ -0.01  │
│ 1503971700 ┆ 348.29  ┆ 348.29  ┆ 348.29  ┆ 348.29  ┆ 0.018749   ┆ 0.0    │
│ 1503588900 ┆ 322.12  ┆ 322.99  ┆ 322.12  ┆ 322.9   ┆ 21.942163  ┆ 0.78   │
│ …          ┆ …       ┆ …       ┆ …       ┆ …       ┆ …          ┆ …      │
│ 1710975900 ┆ 3510.0  ┆ 3526.2  ┆ 3509.5  ┆ 3525.6  ┆ 30.648152  ┆ 15.6   │
│ 1660662600 ┆ 1875.6  ┆ 1881.3  ┆ 1874.4  ┆ 1879.8  ┆ 1

In [119]:
mintime = eth1.select(pl.min("timestamp"))

print(mintime)

#this is important so we can merge two csv's as eth1 runs only from 2017 and we need data from earlier dates.

shape: (1, 1)
┌────────────┐
│ timestamp  │
│ ---        │
│ i64        │
╞════════════╡
│ 1502917500 │
└────────────┘


In [121]:
eth2 = pl.read_csv("ETH_1min.csv", infer_schema_length=2000)\
    .select(pl.col("Unix Timestamp"), pl.col("Open"), pl.col("High"), pl.col("Low"), pl.col("Close"), pl.col("Volume"))\
    .fill_nan(None)\
    .drop_nulls()\
    .with_columns(Return = pl.col("Close")-pl.col("Open"))

print(eth2)

#load eth2 csv and display

shape: (1_984_012, 7)
┌────────────────┬────────┬────────┬────────┬────────┬────────────┬────────┐
│ Unix Timestamp ┆ Open   ┆ High   ┆ Low    ┆ Close  ┆ Volume     ┆ Return │
│ ---            ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---        ┆ ---    │
│ i64            ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64        ┆ f64    │
╞════════════════╪════════╪════════╪════════╪════════╪════════════╪════════╡
│ 1586995200000  ┆ 152.94 ┆ 152.94 ┆ 151.33 ┆ 151.33 ┆ 112.724931 ┆ -1.61  │
│ 1586995140000  ┆ 152.94 ┆ 152.94 ┆ 152.94 ┆ 152.94 ┆ 0.0        ┆ 0.0    │
│ 1586995080000  ┆ 153.17 ┆ 153.17 ┆ 152.94 ┆ 152.94 ┆ 0.013011   ┆ -0.23  │
│ 1586995020000  ┆ 152.96 ┆ 153.2  ┆ 152.96 ┆ 153.17 ┆ 466.782278 ┆ 0.21   │
│ 1586994960000  ┆ 153.14 ┆ 153.14 ┆ 152.96 ┆ 152.96 ┆ 167.930714 ┆ -0.18  │
│ …              ┆ …      ┆ …      ┆ …      ┆ …      ┆ …          ┆ …      │
│ 1462800960     ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 0.375      ┆ 0.0    │
│ 1462800900     ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆

In [78]:
mintime2 = eth2.select(pl.min("Unix Timestamp"))

print(mintime2)

#this is for checking only

shape: (1, 1)
┌────────────────┐
│ Unix Timestamp │
│ ---            │
│ i64            │
╞════════════════╡
│ 1462800720     │
└────────────────┘


In [117]:
filter = eth2.filter(pl.col("Unix Timestamp") < mintime)

print(filter)

#filters out those >= mintime in eth1 becase it means its past the date we need.

shape: (668_613, 7)
┌────────────────┬────────┬────────┬────────┬────────┬───────────┬────────┐
│ Unix Timestamp ┆ Open   ┆ High   ┆ Low    ┆ Close  ┆ Volume    ┆ Return │
│ ---            ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---       ┆ ---    │
│ i64            ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64       ┆ f64    │
╞════════════════╪════════╪════════╪════════╪════════╪═══════════╪════════╡
│ 1502917440     ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ 1502917380     ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ 1502917320     ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ 1502917260     ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ 1502917200     ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ …              ┆ …      ┆ …      ┆ …      ┆ …      ┆ …         ┆ …      │
│ 1462800960     ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 0.375     ┆ 0.0    │
│ 1462800900     ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 60.0      ┆ 0

In [116]:
neweth = eth2.filter(pl.col("Unix Timestamp") < mintime).select(
    pl.col("Unix Timestamp").alias("timestamp"),
    pl.col("Open").cast(pl.Float64).alias("open"),
    pl.col("High").cast(pl.Float64).alias("high"),
    pl.col("Low").cast(pl.Float64).alias("low"),
    pl.col("Close").cast(pl.Float64).alias("close"),
    pl.col("Volume").cast(pl.Float64).alias("volume"),
    pl.col("Return").cast(pl.Float64).alias("return")
    )

print(neweth)

#for eth1 & eh2 to have the same header

shape: (668_613, 7)
┌────────────┬────────┬────────┬────────┬────────┬───────────┬────────┐
│ timestamp  ┆ open   ┆ high   ┆ low    ┆ close  ┆ volume    ┆ return │
│ ---        ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---       ┆ ---    │
│ i64        ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64       ┆ f64    │
╞════════════╪════════╪════════╪════════╪════════╪═══════════╪════════╡
│ 1502917440 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ 1502917380 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ 1502917320 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ 1502917260 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ 1502917200 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 300.91 ┆ 0.0       ┆ 0.0    │
│ …          ┆ …      ┆ …      ┆ …      ┆ …      ┆ …         ┆ …      │
│ 1462800960 ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 0.375     ┆ 0.0    │
│ 1462800900 ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 60.0      ┆ 0.0    │
│ 1462800840 ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 10

In [122]:
ethcombined = pl.concat([eth1, neweth])

print(ethcombined)

#combine the two csv's 

shape: (1_223_565, 7)
┌────────────┬────────┬────────┬────────┬────────┬────────────┬────────┐
│ timestamp  ┆ open   ┆ high   ┆ low    ┆ close  ┆ volume     ┆ return │
│ ---        ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---        ┆ ---    │
│ i64        ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64        ┆ f64    │
╞════════════╪════════╪════════╪════════╪════════╪════════════╪════════╡
│ 1505631900 ┆ 244.1  ┆ 245.34 ┆ 244.04 ┆ 244.58 ┆ 125.832853 ┆ 0.48   │
│ 1507087800 ┆ 292.84 ┆ 293.28 ┆ 292.84 ┆ 293.28 ┆ 45.554642  ┆ 0.44   │
│ 1507131300 ┆ 292.95 ┆ 292.95 ┆ 292.92 ┆ 292.94 ┆ 24.737537  ┆ -0.01  │
│ 1503971700 ┆ 348.29 ┆ 348.29 ┆ 348.29 ┆ 348.29 ┆ 0.018749   ┆ 0.0    │
│ 1503588900 ┆ 322.12 ┆ 322.99 ┆ 322.12 ┆ 322.9  ┆ 21.942163  ┆ 0.78   │
│ …          ┆ …      ┆ …      ┆ …      ┆ …      ┆ …          ┆ …      │
│ 1462800960 ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 0.375      ┆ 0.0    │
│ 1462800900 ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 10.0   ┆ 60.0       ┆ 0.0    │
│ 1462800840 ┆ 10.0   ┆ 10.0 

In [114]:
ethreturns = ethcombined.select("return")

print("Shapiro-Wilk Test:", statistic)
print("P-value:", p_value)

statistic, p_value = shapiro(np.squeeze(ethreturns.to_numpy() ) )

alpha = 0.05
if p_value < alpha:
    print("Reject null hypothesis: Data is not normally distributed")
else:
    print("Fail to reject null hypothesis: Data may be normally distributed")

Shapiro-Wilk Test: 0.4246952533721924
P-value: 0.0
Reject null hypothesis: Data is not normally distributed
